In [2]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

In [3]:
import correlation_tool

# Axioms to test

Given a 7 pip movement of the 'base' symbols:

- If USDJPY, USDCHF rise, EURUSD falls 
- If GBPUSD and GBPJPY rise, EURGBP falls
- If AUDSD falls and USDCAD rises, Gold increases 
- If EURUSD rises and USDCHF falls, EURCHF increases 

## Notes

- Could there be a time-lagged correlation with the above? For example, instead of the 3rd symbol increasing/decreasing based on the first two symbols on the next candle, could the ith element increase instead? So if USDJPY and USDCHF rise, would EURUSD fall on the 6th candle following?

In [117]:
#USDJPY, USDCHF VS EURUSD
UJ = correlation_tool.get_rates('USDJPY.a', mt5.TIMEFRAME_M15, 5000)
UJ['UJ_Return'] = (UJ['open'] - UJ['close']) / 100
UC = correlation_tool.get_rates('USDCHF.a', mt5.TIMEFRAME_M15, 5000)
UC['UC_Return'] = (UC['open'] - UC['close']) 
df = pd.DataFrame(UJ['UJ_Return'])
df2 = pd.DataFrame(UC['UC_Return'])
comb_df = pd.concat([df, df2], axis = 1)
pip_threshold = 0.0005
filtered_df = comb_df[((comb_df['UJ_Return'] > pip_threshold) | (comb_df['UJ_Return'] < -pip_threshold))
                      & ((comb_df['UC_Return'] > pip_threshold) | (comb_df['UC_Return'] < -pip_threshold))]
# Print the filtered dataframe
filtered_df.head(10)

,UJ_Return,UC_Return
time,,
2023-04-28 15:45:00,0.00063,0.00106
2023-04-28 16:00:00,0.00190,0.00216
2023-04-28 16:15:00,-0.00111,-0.00081
2023-04-28 16:30:00,-0.00168,-0.00152
2023-04-28 16:45:00,-0.00063,0.00072
2023-04-28 17:00:00,-0.00170,0.00110
2023-04-28 17:15:00,-0.00140,0.00065
2023-04-28 17:30:00,0.00178,0.00152
2023-04-28 17:45:00,0.00181,0.00080


In [118]:
EU = correlation_tool.get_rates('EURUSD.a', mt5.TIMEFRAME_M15, 5000)
EU['EU_Return'] = (EU['open'] - EU['close'])
EU

,open,high,low,close,EU_Return
time,,,,,
2023-04-28 13:30:00,1.09939,1.09941,1.09884,1.09926,0.00013
2023-04-28 13:45:00,1.09925,1.09926,1.09824,1.09839,0.00086
2023-04-28 14:00:00,1.09839,1.09861,1.09812,1.09850,-0.00011
2023-04-28 14:15:00,1.09846,1.09884,1.09837,1.09879,-0.00033
2023-04-28 14:30:00,1.09879,1.09880,1.09809,1.09812,0.00067
...,...,...,...,...,...
2023-07-11 14:15:00,1.10032,1.10044,1.10000,1.10005,0.00027
2023-07-11 14:30:00,1.10005,1.10007,1.09889,1.09889,0.00116
2023-07-11 14:45:00,1.09889,1.09896,1.09830,1.09837,0.00052


In [119]:
new_df = pd.DataFrame()
for i in EU.index:
    if i in filtered_df.index:
        eu_row = EU.loc[[i]]
        eu = eu_row['EU_Return']
        new_df = EU.loc[filtered_df.index, 'EU_Return']
        new_df = new_df.to_frame()  # Convert the Series to a DataFrame
# Append the row to the 'filtered_df'
final_df = pd.concat([filtered_df, new_df], axis = 1)

#Print the updated 'filtered_df'
final_df

# This df represents the return of EURUSD whenever USDJPY and USDCHF both rise by a value of more than 6 pips.

,UJ_Return,UC_Return,EU_Return
time,,,
2023-04-28 15:45:00,0.00063,0.00106,-0.00128
2023-04-28 16:00:00,0.00190,0.00216,-0.00288
2023-04-28 16:15:00,-0.00111,-0.00081,0.00070
2023-04-28 16:30:00,-0.00168,-0.00152,0.00099
2023-04-28 16:45:00,-0.00063,0.00072,-0.00046
...,...,...,...
2023-07-10 17:00:00,0.00100,0.00195,-0.00127
2023-07-10 17:30:00,-0.00112,-0.00063,0.00038
2023-07-11 09:15:00,0.00123,0.00119,-0.00074


In [136]:
t1 = final_df
t1['EU_Return_Lagged'] = t1['EU_Return'].shift(periods = 1)
t1['UJ+UC'] = t1['UJ_Return'] + t1['UC_Return']
t1.corr()

,UJ_Return,UC_Return,EU_Return,EU_Return_Lagged,UJ+UC
UJ_Return,1.000000,0.737899,-0.694780,-0.027534,0.960424
UC_Return,0.737899,1.000000,-0.879644,-0.016827,0.896687
EU_Return,-0.694780,-0.879644,1.000000,0.011644,-0.818734
EU_Return_Lagged,-0.027534,-0.016827,0.011644,1.000000,-0.025005
UJ+UC,0.960424,0.896687,-0.818734,-0.025005,1.000000
